In [11]:
import pickle

In [13]:
def analyze(path):
    with open("{}.pkl".format(path), "rb+") as file:
        data = pickle.load(file)

    # resort
    for seed, epi_data in data.items():
        for obs_index, obs_ret in epi_data.items():
            if obs_index == "seed":
                continue
            resort = sorted(obs_ret, key=lambda x: x["neuron"]["neuron_index"], reverse=True)
            resort = sorted(resort, key=lambda x: x["neuron"]["layer"], reverse=True)
            epi_data[obs_index] = resort

            # average
    averaged_ret = {}
    for seed, epi_data in data.items():
        for obs, obs_ret in epi_data.items():
            if obs == "seed":
                continue
            if obs in averaged_ret:
                for k,each_neuron in enumerate(obs_ret):
                    assert averaged_ret[obs][k]["layer"] == each_neuron["neuron"]["layer"] and averaged_ret[obs][k]["neuron_index"] == each_neuron["neuron"]["neuron_index"], "{} {}".format(averaged_ret[obs][k]["neuron_index"], each_neuron["neuron"]["neuron_index"])
                    averaged_ret[obs][k]["freq_diff"] = (averaged_ret[obs][k]["freq_diff"]*averaged_ret[obs][k]["seed_num"] + each_neuron["error"]["freq_diff"])/(averaged_ret[obs][k]["seed_num"]+1)
                    averaged_ret[obs][k]["base_phase_diff"] = (averaged_ret[obs][k]["base_phase_diff"]*averaged_ret[obs][k]["seed_num"] + each_neuron["error"]["base_phase_diff"])/(averaged_ret[obs][k]["seed_num"]+1)
                    averaged_ret[obs][k]["seed_num"]+=1
            else:
                averaged_ret[obs] = []
                for each_neuron in obs_ret:
                    averaged_ret[obs].append({"layer": each_neuron["neuron"]["layer"],
                                       "neuron_index": each_neuron["neuron"]["neuron_index"],
                                       "freq_diff": each_neuron["error"]["freq_diff"],
                                       "base_phase_diff": each_neuron["error"]["base_phase_diff"],
                                       "seed_num": 1})
    for obs_dim, avg_ret in averaged_ret.items():
        averaged_ret[obs_dim] = sorted(avg_ret, key=lambda x:x["freq_diff"],reverse=False)
    return averaged_ret

In [10]:
# For anymal:
# 5: angular velocity (yaw)
averaged_ret = analyze()"anymal_ret")
print("angular velocity related neuron:", averaged_ret[5][1]) # 1 better than 0 due to a higher correlation
# 1: x-linear velocity
print("x-linear velocity related neuron:", averaged_ret[1][1]) # 1 better than 0 due to a higher correlation

angular velocity related neuron: {'layer': 0, 'neuron_index': 67, 'freq_diff': 29.66586259745528, 'base_phase_diff': 0.7112072881609092, 'seed_num': 20}
x-linear velocity related neuron: {'layer': 1, 'neuron_index': 17, 'freq_diff': 226.9546785853177, 'base_phase_diff': 0.9976190475527915, 'seed_num': 20}
